# Using scvi-hub to upload pretrained scvi-tools models
In this tutorial, we will see how to use scvi-tools to upload pretrained models onto Hugging Face. We will also see how to handle large training datasets.

If you have not already, make sure to refer to our scvi_hub_into_and_download tutorial, which is a pre-requisite to this one. It introduces Hugging Face (HF) and the scvi-hub, and describes how to use them for downloading pre-trained models from the HF Model Hub.

# Imports
Let's start by adding the Python imports we need.

In [1]:
import scvi
from scvi.hub import HubModel, HubMetadata, HubModelCardHelper

# Pretrain a demo model

Let's pretrain a model on some synthetic data which we'll later use to upload to the scvi-hub later.

In [5]:
local_dir = "local/scvi_hub_upload"

adata = scvi.data.synthetic_iid()
scvi.model.SCVI.setup_anndata(adata)
model = scvi.model.SCVI(adata)
model.train(1)
model.save(local_dir, save_anndata=True, overwrite=True)

/Users/valehvpa/GitRepos/scvi-tools/scvi/data/_built_in_data/_synthetic.py:32: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = AnnData(data)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniconda/base/envs/scvi-hub/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniconda/base/envs/scvi-hub/lib/python3.9

Epoch 1/1: 100%|██████████| 1/1 [00:00<00:00, 39.55it/s, loss=333, v_num=1]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████| 1/1 [00:00<00:00, 31.65it/s, loss=333, v_num=1]


# Model Card and Metadata

To upload pretrained models, you'll need to create an instance of the `HubModel` class and then simply call its `push_to_huggingface_hub` method.

As you can see from the API reference, the `HubModel` init function requires metadata and a Model Card. There are a few ways you can provide these:
- The metadata can be either an instance of the `HubMetadata` class that contains the required metadata for this model, or a path to a JSON file on disk where this metadata can be read from.
- The Model Card can be either an instance of the `HubModelCardHelper` class created for this model, or an instance of the HF [Model Card](https://huggingface.co/docs/huggingface_hub/package_reference/cards#huggingface_hub.ModelCard) object, or a path to a Markdown file on disk where the model card can be read from.
    - You can also use the `HubModelCardHelper` class to create a Model Card from the scvi-tools template, then save it on disk and change it as you wish before passing it into the `HubModel` class.

Here we'll see how to create the HubMetadata and a Model Card from the data on disk.

In [17]:
hm = HubMetadata.from_dir(local_dir, anndata_version="0.8.0")

hmch = HubModelCardHelper.from_dir(
    local_dir,
    license_info="cc-by-4.0",
    anndata_version="0.8.0",
    data_modalities=["rna", "protein"],
    data_is_annotated=False,
    description="This is a demo model used during upload demo.",
    references="None.",
)

INFO     File local/scvi_hub_upload/model.pt already downloaded                 
INFO     File local/scvi_hub_upload/model.pt already downloaded                 


In [18]:
print(hmch.model_card.content)

---
license: cc-by-4.0
library_name: scvi-tools
tags:
- model_cls_name:SCVI
- scvi_version:0.19.0a0
- anndata_version:0.8.0
- modality:rna
- modality:protein
- annotated:False
---

# Description

This is a demo model used during upload demo.

# Model properties

Many model properties are in the model tags. Some more are listed below.

**model_init_params**:
```json
{
    "n_hidden": 128,
    "n_latent": 10,
    "n_layers": 1,
    "dropout_rate": 0.1,
    "dispersion": "gene",
    "gene_likelihood": "zinb",
    "latent_distribution": "normal"
}
```

**model_setup_anndata_args**:
```json
{
    "layer": null,
    "batch_key": null,
    "labels_key": null,
    "size_factor_key": null,
    "categorical_covariate_keys": null,
    "continuous_covariate_keys": null
}
```

**model_summary_stats**:
|     Summary Stat Key     | Value |
|--------------------------|-------|
|         n_batch          |   1   |
|         n_cells          |  400  |
| n_extra_categorical_covs |   0   |
| n_extra_conti

Suppose I wanted to change the References section to say "None. This is a demo model.". To do that, I'll save the card to disk and then change it manually.

In [21]:
hmch.model_card.save("local/my_model_card.md") # then change the markdown file on disk...

In [9]:
print(hmch.model_card.content)

'---\nlicense: cc-by-4.0\nlibrary_name: scvi-tools\ntags:\n- model_cls_name:SCVI\n- scvi_version:0.19.0a0\n- anndata_version:0.8.0\n- modality:rna\n- modality:protein\n- annotated:False\n---\n\n# Description\n\nThis is a demo model used during upload demo.\n\n# Model properties\n\nMany model properties are in the model tags. Some more are listed below.\n\n**model_init_params**:\n```json\n{\n    "n_hidden": 128,\n    "n_latent": 10,\n    "n_layers": 1,\n    "dropout_rate": 0.1,\n    "dispersion": "gene",\n    "gene_likelihood": "zinb",\n    "latent_distribution": "normal"\n}\n```\n\n**model_setup_anndata_args**:\n```json\n{\n    "layer": null,\n    "batch_key": null,\n    "labels_key": null,\n    "size_factor_key": null,\n    "categorical_covariate_keys": null,\n    "continuous_covariate_keys": null\n}\n```\n\n**model_summary_stats**:\n|     Summary Stat Key     | Value |\n|--------------------------|-------|\n|         n_batch          |   1   |\n|         n_cells          |  400  |\n|